In [1]:
# Importing necessary libraries
import pandas as pd
import numpy as np
import re

In [2]:
df = pd.read_csv('scraped_df.csv')
df

,Society,Card Title,Carpet Area,Price,Price Per Sqft,Floor
0,Gagan Aviva,"2 BHK Flat for Sale in Gagan Aviva, Kesnand,...",356 sqft,₹42 Lac,"₹7,970 per sqft",NaN
1,Codename Plus Life,"3 BHK Flat for Sale in Codename Plus Life, K...",NaN,₹86.4 Lac,"₹9,736 per sqft",NaN
2,Chandrakamal,"2 BHK Flat for Sale in Chandrakamal, Shukraw...",938 sqft,₹1.75 Cr,"₹13,823 per sqft",NaN
3,MJ Opera,"2 BHK Flat for Sale in MJ Opera, Wakad, Pune",816 sqft,₹90.9 Lac,"₹7,796 per sqft",NaN
4,ANP Autograph,"4 BHK Flat for Sale in ANP Autograph, Punawa...",1777 sqft,₹2.22 Cr,"₹10,039 per sqft",NaN
...,...,...,...,...,...,...
2984,NaN,1 BHK Apartment for Sale in Lohegaon Pune,NaN,₹37.7 Lac,"₹5,994 per sqft",6 out of 6
2985,Gera World of Joy,"1 BHK Apartment for Sale in Gera World of Joy,...",242 sqft,₹40 Lac,"₹13,333 per sqft",6 out of 24
2986,Vanaha Springs,"2 BHK Apartment for Sale in Vanaha Springs, Ba...",801 sqft,₹85.5 Lac,NaN,6 out of 20
2987,Godrej 24,"Apartment for Sale in Godrej 24, Hinjawadi Pune",NaN,₹86.5 Lac,NaN,NaN


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2989 entries, 0 to 2988
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Society         2421 non-null   object
 1   Card Title      2989 non-null   object
 2   Carpet Area     2560 non-null   object
 3   Price           2989 non-null   object
 4   Price Per Sqft  2771 non-null   object
 5   Floor           2842 non-null   object
dtypes: object(6)
memory usage: 140.2+ KB


In [4]:
def convert_price_to_number(price):
    price = price.replace('₹', '').replace(',', '').strip()
    if 'Cr' in price:
        return int(float(price.replace('Cr', '')) * 10000000)
    elif 'Lac' in price:
        return int(float(price.replace('Lac', '')) * 100000)
    else:
        return int(price)

# Applying the function to the 'Price' column in the DataFrame
df['Price'] = df['Price'].apply(convert_price_to_number)

In [5]:
# Function to extract and convert numerical values to integers in 'Carpet Area' column
def extract_and_convert_carpet_area(carpet):
    if isinstance(carpet, str):
        match = re.search(r'\d+', carpet)
        if match:
            return int(match.group())
    return np.nan  # Returning NaN for non-string values or cases where no digits are found

# converting to int
df['Carpet Area'] = df['Carpet Area'].apply(extract_and_convert_carpet_area).astype('Int64')

In [6]:
# Renaming the 'Floor' column to 'total_floors'
df.rename(columns={'Floor': 'total_floors'}, inplace=True)

# function to extract the value after 'out of'
def extract_total_floors(floor):
    if isinstance(floor, str):
        out_of_index = floor.find('out of')
        if out_of_index != -1:
            return int(floor[out_of_index + 7:])
    return None

# Apply the function to create the 'total_floors' column
df['total_floors'] = df['total_floors'].apply(extract_total_floors)

# Converting to int
df['total_floors'] = df['total_floors'].astype('Int64')

In [7]:
# function to remove 'per sqft' and '₹' symbol
def remove_per_sqft(price):
    if isinstance(price, str):
        return price.replace(' per sqft', '').replace('₹', '')
    return None

# Applying the function to the 'Price Per Sqft' column
df['Price Per Sqft'] = df['Price Per Sqft'].apply(remove_per_sqft)

# function to remove commas and convert to integers
def remove_commas_and_convert_to_int(price):
    if isinstance(price, str):
        # Remove commas and convert to integer
        return int(price.replace(',', ''))
    return None

# Applying the function to the 'Price Per Sqft' column
df['Price Per Sqft'] = df['Price Per Sqft'].apply(remove_commas_and_convert_to_int)

# convert ti int
df['Price Per Sqft'] = df['Price Per Sqft'].astype('Int64')

In [8]:
# Define a function to extract BHK information
def extract_bhk(card_title):
    match = re.search(r'(\d+)\sBHK', card_title)
    if match:
        return int(match.group(1))
    return None

# Create a new column 'BHK' using the extracted BHK information
df['BHK'] = df['Card Title'].apply(extract_bhk)

#converting to int
df['BHK'] = df['BHK'].astype('Int64')

In [9]:
# function to extract location information
def extract_location(card_title):
    location = card_title.split('Sale in')[1].split('Pune')[0].strip(',')
    return location.rstrip(',').strip()

# Creating a new column 'Location'
df['Location'] = df['Card Title'].apply(extract_location)

In [10]:
# Calculate 'Super Area' by dividing 'Price' by 'Price Per Sqft' and rounding to the closest integer
df['Super Area'] = (df['Price'] / df['Price Per Sqft']).round().astype('Int64')

In [11]:
# Rename the 'Society' column to 'Project'
df.rename(columns={'Society': 'Project'}, inplace=True)

In [12]:
df

,Project,Card Title,Carpet Area,Price,Price Per Sqft,total_floors,BHK,Location,Super Area
0,Gagan Aviva,"2 BHK Flat for Sale in Gagan Aviva, Kesnand,...",356,4200000,7970,<NA>,2,"Gagan Aviva, Kesnand,",527
1,Codename Plus Life,"3 BHK Flat for Sale in Codename Plus Life, K...",<NA>,8640000,9736,<NA>,3,"Codename Plus Life, Kondhwa,",887
2,Chandrakamal,"2 BHK Flat for Sale in Chandrakamal, Shukraw...",938,17500000,13823,<NA>,2,"Chandrakamal, Shukrawar Peth,",1266
3,MJ Opera,"2 BHK Flat for Sale in MJ Opera, Wakad, Pune",816,9090000,7796,<NA>,2,"MJ Opera, Wakad,",1166
4,ANP Autograph,"4 BHK Flat for Sale in ANP Autograph, Punawa...",1777,22200000,10039,<NA>,4,"ANP Autograph, Punawale, Pimpri Chinchwad,",2211
...,...,...,...,...,...,...,...,...,...
2984,NaN,1 BHK Apartment for Sale in Lohegaon Pune,<NA>,3770000,5994,6,1,Lohegaon,629
2985,Gera World of Joy,"1 BHK Apartment for Sale in Gera World of Joy,...",242,4000000,13333,24,1,"Gera World of Joy, Kharadi",300
2986,Vanaha Springs,"2 BHK Apartment for Sale in Vanaha Springs, Ba...",801,8550000,<NA>,20,2,"Vanaha Springs, Bavdhan",<NA>
2987,Godrej 24,"Apartment for Sale in Godrej 24, Hinjawadi Pune",<NA>,8650000,<NA>,<NA>,<NA>,"Godrej 24, Hinjawadi",<NA>


In [13]:
# function to modify the 'Location' column
def modify_location(location):
    parts = location.split(',')
    if len(parts) > 1:
        return parts[-2].strip()
    else:
        return parts[-1].strip(',')

# Apply the function to the 'Location' column
df['Location'] = df['Location'].apply(modify_location)
df

,Project,Card Title,Carpet Area,Price,Price Per Sqft,total_floors,BHK,Location,Super Area
0,Gagan Aviva,"2 BHK Flat for Sale in Gagan Aviva, Kesnand,...",356,4200000,7970,<NA>,2,Kesnand,527
1,Codename Plus Life,"3 BHK Flat for Sale in Codename Plus Life, K...",<NA>,8640000,9736,<NA>,3,Kondhwa,887
2,Chandrakamal,"2 BHK Flat for Sale in Chandrakamal, Shukraw...",938,17500000,13823,<NA>,2,Shukrawar Peth,1266
3,MJ Opera,"2 BHK Flat for Sale in MJ Opera, Wakad, Pune",816,9090000,7796,<NA>,2,Wakad,1166
4,ANP Autograph,"4 BHK Flat for Sale in ANP Autograph, Punawa...",1777,22200000,10039,<NA>,4,Pimpri Chinchwad,2211
...,...,...,...,...,...,...,...,...,...
2984,NaN,1 BHK Apartment for Sale in Lohegaon Pune,<NA>,3770000,5994,6,1,Lohegaon,629
2985,Gera World of Joy,"1 BHK Apartment for Sale in Gera World of Joy,...",242,4000000,13333,24,1,Gera World of Joy,300
2986,Vanaha Springs,"2 BHK Apartment for Sale in Vanaha Springs, Ba...",801,8550000,<NA>,20,2,Vanaha Springs,<NA>
2987,Godrej 24,"Apartment for Sale in Godrej 24, Hinjawadi Pune",<NA>,8650000,<NA>,<NA>,<NA>,Godrej 24,<NA>


In [14]:
# reordering the data set and dropping the unnecessary column(s)

df.drop(columns=['Card Title'], inplace=True)

new_order = ['Project', 'BHK', 'Location', 'total_floors', 'Carpet Area', 'Super Area', 'Price', 'Price Per Sqft']
df = df.reindex(columns=new_order)

In [15]:
# Drop rows with any missing values
df.dropna(inplace=True)

# Reset the index to arrange it properly after dropping missing values
df.reset_index(drop=True, inplace=True)

df


,Project,BHK,Location,total_floors,Carpet Area,Super Area,Price,Price Per Sqft
0,Blue Ridge,2,Blue Ridge,25,867,1145,9500000,8297
1,57 Elevate,3,57 Elevate,35,1010,1364,10000000,7331
2,Goel Ganga Avanta,3,Goel Ganga Avanta,30,1060,1480,14000000,9459
3,Shubh Tristar,3,Koregaon Park Annexe,45,1268,1650,16000000,9699
4,Basil Maximus,2,Punawale,19,749,974,6700000,6879
...,...,...,...,...,...,...,...,...
1821,Sahara Royal,2,Sahara Royal,4,980,1102,4950000,4490
1822,Mantra 7 Hills,3,Mantra 7 Hills,10,600,803,5500000,6849
1823,Kolte Patil Ivy Estate,3,Kolte Patil Ivy Estate,12,1055,1716,12500000,7284
1824,Goodwill Metropolis West,1,Goodwill Metropolis West,11,480,560,4400000,7857


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1826 entries, 0 to 1825
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Project         1826 non-null   object
 1   BHK             1826 non-null   Int64 
 2   Location        1826 non-null   object
 3   total_floors    1826 non-null   Int64 
 4   Carpet Area     1826 non-null   Int64 
 5   Super Area      1826 non-null   Int64 
 6   Price           1826 non-null   int64 
 7   Price Per Sqft  1826 non-null   Int64 
dtypes: Int64(5), int64(1), object(2)
memory usage: 123.2+ KB
